# Credit Card Fraud Detection
Rule-based → Machine Learning approach


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

%matplotlib inline

In [ ]:
import os

os.listdir()


In [ ]:
!mv /creditcard.csv /content/


In [ ]:
!pwd
!ls /content


In [ ]:
df = pd.read_csv("creditcard.csv")

df.head()

In [ ]:
df.shape

# analyzing the dataset here rows will be the no of
# transactions and columns will be be features + target

In [ ]:
df.columns


In [ ]:
df.info()


### Missing Values Analysis

The dataset contains no missing (null) values across all features, as confirmed using `df.info()`.
Therefore, no imputation or missing-value handling is required for this dataset.


In [ ]:
df['Class'].value_counts()
df['Class'].value_counts(normalize=True) * 100

### Class Imbalance Analysis

The target variable `Class` shows a highly imbalanced distribution, where fraudulent transactions (Class = 1) represent a very small percentage of the total dataset, while non-fraudulent transactions (Class = 0) dominate.

This imbalance makes accuracy an unreliable metric, as a model could predict all transactions as non-fraud and still achieve high accuracy. Therefore, evaluation metrics such as precision, recall, and F1-score are more appropriate for this problem.


In [ ]:
#now visulization of the percentage distributions of the target "class"

sns.countplot(x='Class', data=df)
plt.title('class distribution (Fraud vs Non Fraud)')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df[df['Class']== 0]['Amount'], bins=50, log_scale=True, label='Non-Fraud', color='blue')
sns.histplot(df[df['Class']== 1]['Amount'], bins=50, log_scale=True, label = 'Fraud', color='red')
plt.title('Distribution of Transaction Amounts (Fraud vs Non-Fraud)')
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency -> (log scale)')
plt.show()

From the transaction amount distribution, we observe a highly right-skewed pattern with a long tail of extreme values. This indicates the presence of outliers and non-normality, which can negatively affect many machine learning models if not handled properly. The log scale helps reveal structure across different magnitudes. While amount alone does not clearly separate fraud from non-fraud, it remains an important supporting feature when combined with others.

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='Class', y='Amount', data=df)
plt.yscale('log')
plt.title('Distribution of Transaction Amounts by Class (log scale)')
plt.show()

Do fraud transactions generally involve higher amounts?
Answer:  ❌No — median is lower

Are fraud transactions more variable?
Answer: ✅ Yes — larger spread

Can amount alone classify fraud?
Answer: ❌ No — heavy overlap

Is amount still useful as a feature?
Answer: ✅ Yes — but after transformation and with other features

*******
Transaction amount is skewed, contains outliers, and overlaps across classes. Therefore, we should transform it (log), avoid rule-based thresholds, and combine it with other features using models robust to imbalance.

In [ ]:
#feature engineering ->creating a transformed class
df['Log_Amount'] = np.log1p(df['Amount'])


In [ ]:
df[['Amount', 'Log_Amount']].describe()


In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df[df['Class']==0]['Log_Amount'], bins=50, label='Non-Fraud', log_scale=False)
sns.histplot(df[df['Class']==1]['Log_Amount'], bins=50, label='Fraud', color='red', log_scale=False)
plt.legend()
plt.title("Log_Amount Distribution by Class")
plt.show()

Is Log_Amount alone sufficient to classify fraud?

Answer: ❌ No
But it helps models learn patterns better when combined with others.

In [ ]:
#prepration for modeling
X= df.drop(columns=['Class'])
y=df['Class']

In [ ]:
#we will be using the train test split as the imbalance is extreme!

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


BASE MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression

baseline_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

baseline_model.fit(X_train_scaled, y_train)


In [ ]:
y_pred = baseline_model.predict(X_test_scaled)


In [ ]:
#model  evaluation
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))



In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

plt.figure()
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Logistic Regression Baseline")
plt.show()

TN | FP
_______
FN | TP





*   TP → fraud caught ✅
*   FN → fraud missed ❌
*   FP → false alarm
*   TN → correct non-fraud





In [ ]:
#Probability Threshold Tuning
y_proba = baseline_model.predict_proba(X_test_scaled)[:, 1]


In [ ]:

threshold = 0.3
y_pred_custom = (y_proba >= threshold).astype(int)
#Instead of predictions, get probabilities

In [ ]:
print(classification_report(y_test, y_pred_custom))


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

plt.figure()
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve")
plt.show()


A MORE STRINGER MODEL

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


In [ ]:
y_pred_rf = rf_model.predict(X_test)

print(classification_report(y_test, y_pred_rf))


| Model          | Recall (Fraud) | Precision | F1 |
| -------------- | -------------- | --------- | -- |
| Logistic (0.5) | ❌              | ❌         | ❌  |
| Logistic (0.3) | ✅              | ⚠️        | ✅  |
| Random Forest  | 🔥             | 🔥        | 🔥 |


using SMOTE to Handle Imbalance

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [ ]:
pd.Series(y_train_smote).value_counts()

**Train Model on SMOTE Data**

**1)**  Logistic Regression on SMOTE

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_smote = LogisticRegression(max_iter=1000)
lr_smote.fit(X_train_smote, y_train_smote)

y_pred_lr_smote = lr_smote.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred_lr_smote))


**2)**  Random Forest (SMOTE)

In [ ]:
rf_smote = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf_smote.fit(X_train_smote, y_train_smote)
y_pred_rf_smote = rf_smote.predict(X_test)

print(classification_report(y_test, y_pred_rf_smote))


**Model Comparison & Hyper parameter Tuning**

In [ ]:
#helper function
from sklearn.metrics import classification_report, roc_auc_score

def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    print(f"\n===== {name} =====")
    print(classification_report(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_prob))


In [ ]:
#logistic regression (Smote)
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_smote, y_train_smote)

evaluate_model("Logistic Regression (SMOTE)", lr, X_test, y_test)


In [ ]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train_smote, y_train_smote)

evaluate_model("Gradient Boosting", gb, X_test, y_test)


In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_smote, y_train_smote)

evaluate_model("Random Forest (SMOTE)", rf, X_test, y_test)


| Model                         | Precision | Recall   | F1   | ROC-AUC   |
| ----------------------------- | --------- | -------- | ---- | --------- |
| Logistic Regression           | low       | low      | low  | baseline  |
| Random Forest (SMOTE)         | **0.82**  | **0.70** | 0.76 | 0.956     |
| **Gradient Boosting (SMOTE)** | **0.13**  | **0.88** | 0.23 | **0.977** |


**Gradient Boosting**

Best recall → catches most frauds

Best ROC-AUC → best ranking ability

Lower precision → more false alarms (acceptable in fraud)

👉 In fraud detection:

**Missing a fraud is worse than flagging a normal transaction Gradient Boosting is a defensible final choice**

### Model Selection Summary

Three models were evaluated: Logistic Regression, Random Forest, and Gradient Boosting.
Due to extreme class imbalance, recall and ROC-AUC were prioritized over accuracy.

Gradient Boosting achieved the highest fraud recall (0.88) and ROC-AUC (0.97),
making it the preferred model for this problem despite lower precision.
Hyperparameter tuning was not performed due to computational constraints and
is considered future work.
